# LangChain 中的储存(Memory)模块
LangChain 提供了多种储存类型。其中，缓冲区储存允许保留最近的聊天消息，摘要储存则提供了对整个对话的摘要。实体储存则允许在多轮对话中保留有关特定实体的信息。这些记忆组件都是模块化的，可与其他组件组合使用，从而增强机器人的对话管理能力。储存模块可以通过简单的 API 调用来访问和更新，允许开发人员更轻松地实现对话历史记录的管理和维护。

该文档主要介绍两种存储模块
- 对话缓存储存 (ConversationBufferMemory）
- 对话缓存窗口储存 (ConversationBufferWindowMemory）

在 LangChain 中，储存指的是大语言模型（LLM）的短期记忆，仅在当次对话中进行存储。

## 对话缓存存储
进行多轮对话

In [4]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains.conversation.base import ConversationChain
from langchain.memory import ConversationBufferMemory


# 初始化对话模型
llm = ChatOpenAI(
    model="qwen-turbo",
    api_key=os.getenv("API_KEY"),
    base_url=os.getenv("API_BASEURL")
)

memory = ConversationBufferMemory()

chain = ConversationChain(llm = llm,memory = memory,verbose = True)


# 进行多轮对话
word1 = "你好，我是呱呱蛙。"

word2 = "你认识成龙吗？ 说一部他出演的电影。 只说电影名，不要说其他内容。"

word3 = "你还记得我叫什么吗？"

words = [word1,word2,word3]

# 模拟进行多轮对话

for word in words:
    response = chain.predict(input = word)
    print(f"对话内容：{word}")
    print(f"对话回复：{response}")
    

print("="*20)

# 查看存储缓存

buffer = memory.load_memory_variables({})

print(f"存储：{buffer}")


# 手动添加对话内容
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})

# 查看存储缓存
buffer = memory.load_memory_variables({})

print(f"存储：{buffer}")








> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好，我是呱呱蛙。
AI:

> Finished chain.
对话内容：你好，我是呱呱蛙。
对话回复：你好呱呱蛙！很高兴认识你。我是通义千问，你可以叫我Qwen。我是一个大型语言模型，能够帮助你回答问题、创作文字，如写故事、公文、技术文档等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我哦！你今天过得怎么样？


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: 你好，我是呱呱蛙。
AI: 你好呱呱蛙！很高兴认识你。我是通义千问，你可以叫我Qwen。我是一个大型语言模型，能够帮助你回答问题、创作文字，如写故事、公文、技术文档等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我哦！你今天过得怎么样？
Human: 你认识成龙吗？ 说一部他出演的电影。 只说电影名，不要说其他内容。
AI:



## 对话缓存窗口存储
随着对话变得越来越长，所需的内存量也变得非常长。将大量的tokens发送到LLM的成本，也会变得更加昂贵，这也就是为什么API的调用费用，通常是基于它需要处理的tokens数量而收费的。

针对以上问题，LangChain也提供了几种方便的储存方式来保存历史对话。其中，对话缓存窗口储存只保留一个窗口大小的对话。它只使用最近的n次交互。这可以用于保持最近交互的滑动窗口，以便缓冲区不会过大。

In [5]:
from langchain.memory import ConversationBufferWindowMemory

# k=1表明只保留最后的一个对话记忆
memory = ConversationBufferWindowMemory(k=1)  
memory.save_context({"input": "你好，我叫呱呱蛙"}, {"output": "你好呱呱蛙！很高兴认识你。我是通义千问，你可以叫我Qwen。我是一个大型语言模型，能够帮助你回答问题、创作文字，如写故事、公文、技术文档等，还能表达观点，玩游戏等。如果你有任何问题或需要帮助，尽管告诉我哦！你今天过得怎么样？"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.load_memory_variables({})

C:\Users\Administrator\AppData\Local\Temp\ipykernel_30672\1218610375.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=1)


{'history': 'Human: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}